### Import Libraries and connect spark

In [274]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import year,col, when, sum as _sum,round,mean

os.environ["SPARK_LOCAL_IP"] = "127.0.0.1"


try:
    if "spark" in locals():
        spark.stop()  

    spark = (
        SparkSession.builder.master("local[*]")
        .appName("practice")
        .config("spark.driver.host", "127.0.0.1")
        .config("spark.driver.bindAddress", "127.0.0.1")
        .getOrCreate()
    )

    print("Spark is successfully connected!")
    spark.sparkContext.setLogLevel("ERROR")

except Exception as e:
    print(f"Failed to start Spark: {e}")

Spark is successfully connected!


In [275]:
# load data
df = spark.read.csv("../data/Supplement_Sales_Weekly_Expanded.csv", header=True,inferSchema=True)
df.show()


+----------+------------------+-----------+----------+-----+-------+--------+--------------+--------+--------+
|      Date|      Product Name|   Category|Units Sold|Price|Revenue|Discount|Units Returned|Location|Platform|
+----------+------------------+-----------+----------+-----+-------+--------+--------------+--------+--------+
|2020-01-06|      Whey Protein|    Protein|       143|31.98|4573.14|    0.03|             2|  Canada| Walmart|
|2020-01-06|         Vitamin C|    Vitamin|       139|42.51|5908.89|    0.04|             0|      UK|  Amazon|
|2020-01-06|          Fish Oil|      Omega|       161|12.91|2078.51|    0.25|             0|  Canada|  Amazon|
|2020-01-06|      Multivitamin|    Vitamin|       140|16.07| 2249.8|    0.08|             0|  Canada| Walmart|
|2020-01-06|       Pre-Workout|Performance|       157|35.47|5568.79|    0.25|             3|  Canada|   iHerb|
|2020-01-06|              BCAA| Amino Acid|       154|41.19|6343.26|    0.13|             1|      UK| Walmart|
|

In [276]:
#check for nulls
#data = ps.read_csv("../data/Supplement_Sales_Weekly_Expanded.csv")
for c in df.columns:
    null_count = df.filter(col(c).isNull()).count()
    print(f"{c} : {null_count}")

Date : 0
Product Name : 0
Category : 0


Units Sold : 0
Price : 0
Revenue : 0
Discount : 0
Units Returned : 0
Location : 0
Platform : 0


In [277]:
## check data typed
df.dtypes

[('Date', 'date'),
 ('Product Name', 'string'),
 ('Category', 'string'),
 ('Units Sold', 'int'),
 ('Price', 'double'),
 ('Revenue', 'double'),
 ('Discount', 'double'),
 ('Units Returned', 'int'),
 ('Location', 'string'),
 ('Platform', 'string')]

In [278]:
#check table overview
df.describe().show()

+-------+------------+----------+------------------+------------------+------------------+-------------------+------------------+--------+--------+
|summary|Product Name|  Category|        Units Sold|             Price|           Revenue|           Discount|    Units Returned|Location|Platform|
+-------+------------+----------+------------------+------------------+------------------+-------------------+------------------+--------+--------+
|  count|        4384|      4384|              4384|              4384|              4384|               4384|              4384|    4384|    4384|
|   mean|        NULL|      NULL|150.20027372262774|  34.7812294708029| 5226.569445711672|0.12439781021897832| 1.531478102189781|    NULL|    NULL|
| stddev|        NULL|      NULL|12.396099418079803|14.198308740365304|2192.4919458261884|0.07179172702425907|1.2584785259574762|    NULL|    NULL|
|    min| Ashwagandha|Amino Acid|               103|              10.0|            1284.0|                0.0|  

### Analyze the data

In [279]:
## Sales Performance Over Time by year
df.groupBy(year("Date").alias("year")).agg(
    _sum("Units Sold").alias("total_units_sold"), 
    round(_sum("Revenue"),2).alias("total_revenue")).orderBy(col("total_units_sold").desc()).show()

+----+----------------+-------------+
|year|total_units_sold|total_revenue|
+----+----------------+-------------+
|2024|          127707|   4429367.35|
|2022|          125167|   4372808.55|
|2023|          125038|   4470870.75|
|2021|          124701|   4294248.17|
|2020|          124657|   4323393.22|
|2025|           31208|   1022592.41|
+----+----------------+-------------+



In [280]:
## Sales Performance Over Time by month
df.groupBy(sf.monthname("Date").alias("month")).agg(
    sf.sum("Units Sold").alias("total_units_sold"),
    sf.round(sf.sum("Revenue"), 2).alias("total_revenue"),
).orderBy(sf.col("total_revenue").desc()).orderBy(
    sf.col("total_units_sold").desc()
).show()

+-----+----------------+-------------+
|month|total_units_sold|total_revenue|
+-----+----------------+-------------+
|  Mar|           65398|    2271026.4|
|  Jan|           64780|   2171966.05|
|  Feb|           57440|   2073046.84|
|  May|           55125|   1920081.21|
|  Aug|           55122|   1925991.58|
|  Nov|           53078|   1750791.56|
|  Jul|           52938|   1849431.84|
|  Oct|           52679|   1816041.16|
|  Apr|           50644|    1685662.3|
|  Jun|           50555|    1826877.4|
|  Dec|           50515|   1839926.51|
|  Sep|           50204|    1782437.6|
+-----+----------------+-------------+



In [281]:
## Product & Category Performance Analysis
df.groupBy("Product Name").agg(
    sf.round(sf.sum("Units Sold"), 2).alias('Unit Sold'),
    sf.round(sf.sum("Revenue"), 2).alias("total_revenue"),
).orderBy(sf.col("total_revenue").desc()).show()

+------------------+---------+-------------+
|      Product Name|Unit Sold|total_revenue|
+------------------+---------+-------------+
|            Biotin|    41533|   1486798.62|
|              Zinc|    41204|   1482546.95|
|       Pre-Workout|    41287|   1477183.78|
|              BCAA|    41027|   1464819.63|
|          Fish Oil|    41325|   1451065.87|
| Green Tea Extract|    40743|   1440900.05|
| Collagen Peptides|    40856|   1433297.24|
|          Creatine|    41236|    1432518.4|
|   Iron Supplement|    41194|   1431582.41|
|      Whey Protein|    41264|   1422194.85|
|         Vitamin C|    40727|   1421998.07|
|Electrolyte Powder|    41065|   1411951.38|
|       Ashwagandha|    41408|   1405700.79|
|         Melatonin|    41165|   1397315.79|
|      Multivitamin|    41174|   1391427.99|
|         Magnesium|    41270|   1361978.63|
+------------------+---------+-------------+



In [282]:
# Discount Effectiveness Analysis
#Units sold with discounts
df.groupBy("Category").agg(
    _sum(sf.when(col("Discount") > 0, col("Units Sold")).otherwise(0)).alias(
        "discounted_units_sold"
    )
).show()

+-----------+---------------------+
|   Category|discounted_units_sold|
+-----------+---------------------+
|     Herbal|                40421|
| Amino Acid|                40295|
|  Sleep Aid|                39664|
|  Hydration|                39785|
|    Vitamin|               121363|
|    Mineral|               122297|
|    Protein|                80211|
|      Omega|                40443|
|Performance|                80858|
| Fat Burner|                39736|
+-----------+---------------------+



In [283]:
# Units sold without discounts
df.groupBy("Category").agg(
    _sum(sf.when(col("Discount") == 0, col("Units Sold")).otherwise(0)).alias(
        "discounted_units_sold"
    )
).show()


+-----------+---------------------+
|   Category|discounted_units_sold|
+-----------+---------------------+
|     Herbal|                  987|
| Amino Acid|                  732|
|  Sleep Aid|                 1501|
|  Hydration|                 1280|
|    Vitamin|                 2071|
|    Mineral|                 1371|
|    Protein|                 1909|
|      Omega|                  882|
|Performance|                 1665|
| Fat Burner|                 1007|
+-----------+---------------------+



In [284]:
## Total revenue with discount 
df.agg(
    _sum(when(col("Discount") > 0, round(col("Revenue"),2)).otherwise(0)).alias(
        "discounted_revenue"
    ),
    _sum(when(col("Discount") == 0, col("Revenue")).otherwise(0)).alias(
        "non_discounted_revenue"
    ),
).show()


+-------------------+----------------------+
| discounted_revenue|non_discounted_revenue|
+-------------------+----------------------+
|2.244589228999996E7|     467388.1599999998|
+-------------------+----------------------+



In [285]:
#Location & Platform Performance Analysis
df.groupBy('Location').agg(_sum('Revenue').alias('Revenue'),_sum('Units Sold').alias('Unit Sold')).show()

+--------+-----------------+---------+
|Location|          Revenue|Unit Sold|
+--------+-----------------+---------+
|     USA|7360740.380000001|   211188|
|      UK|7703960.339999997|   221237|
|  Canada|7848579.729999992|   226053|
+--------+-----------------+---------+



In [286]:
# Best platform per location by Unit sold
df.groupBy(["Platform", "Location"]).agg(
    _sum("Revenue").alias("Revenue"), _sum("Units Sold").alias("Unit Sold")
).orderBy(col("Revenue").desc()).show()

+--------+--------+------------------+---------+
|Platform|Location|           Revenue|Unit Sold|
+--------+--------+------------------+---------+
|   iHerb|  Canada| 2716096.380000003|    78084|
| Walmart|      UK|2637066.2500000014|    75363|
|   iHerb|      UK| 2624222.859999999|    74238|
|  Amazon|  Canada|2613844.2800000007|    74802|
|  Amazon|     USA|2612936.2700000005|    74185|
| Walmart|  Canada| 2518639.069999999|    73167|
|   iHerb|     USA|2514941.8100000005|    73105|
|  Amazon|      UK| 2442671.230000004|    71636|
| Walmart|     USA| 2232862.299999999|    63898|
+--------+--------+------------------+---------+



In [287]:
#returns by Location
df.groupBy("Location").agg(
    _sum("Units Returned").alias("Units Returned"),
    _sum("Units Sold").alias("Unit Sold"),
).orderBy(col("Units Returned").desc()).show()

+--------+--------------+---------+
|Location|Units Returned|Unit Sold|
+--------+--------------+---------+
|      UK|          2327|   221237|
|  Canada|          2259|   226053|
|     USA|          2128|   211188|
+--------+--------------+---------+



In [288]:
#return rate for supplement
df = df.withColumn(
    "Return Rate", round((col("Units Returned") / col("Units Sold")) * 100, 2))
df.show()

+----------+------------------+-----------+----------+-----+-------+--------+--------------+--------+--------+-----------+
|      Date|      Product Name|   Category|Units Sold|Price|Revenue|Discount|Units Returned|Location|Platform|Return Rate|
+----------+------------------+-----------+----------+-----+-------+--------+--------------+--------+--------+-----------+
|2020-01-06|      Whey Protein|    Protein|       143|31.98|4573.14|    0.03|             2|  Canada| Walmart|        1.4|
|2020-01-06|         Vitamin C|    Vitamin|       139|42.51|5908.89|    0.04|             0|      UK|  Amazon|        0.0|
|2020-01-06|          Fish Oil|      Omega|       161|12.91|2078.51|    0.25|             0|  Canada|  Amazon|        0.0|
|2020-01-06|      Multivitamin|    Vitamin|       140|16.07| 2249.8|    0.08|             0|  Canada| Walmart|        0.0|
|2020-01-06|       Pre-Workout|Performance|       157|35.47|5568.79|    0.25|             3|  Canada|   iHerb|       1.91|
|2020-01-06|    

In [290]:
#platform with highest return rate 
df.groupBy('Platform').agg(round(mean('Return Rate'),2).alias('Return Rate')).show()

+--------+-----------+
|Platform|Return Rate|
+--------+-----------+
| Walmart|        1.0|
|  Amazon|       1.04|
|   iHerb|       1.02|
+--------+-----------+

